In [50]:
import pandas as pd
import re

In [51]:
df = pd.read_csv('/content/samsung_specs.csv')

df.head()

,Model,Release Date,Display Size,Display Res,Battery,Main Camera,Selfie Camera,Memory,Price
0,Samsung Galaxy A07,"2026, January 13","6.7 inches, 108.4 cm2 (~83.6% screen-to-body r...","720 x 1600 pixels, 20:9 ratio (~262 ppi density)",Li-Ion 6000 mAh,"50 MP, f/1.8, (wide), PDAF\nAuxiliary lens","8 MP, f/2.0, 26mm (wide), 1/4.0"", 1.12µm","128GB 4GB RAM, 128GB 6GB RAM",About 150 EUR
1,Samsung Galaxy Z TriFold,"2025, December 02","10.0 inches, 305.4 cm2 (~89.6% screen-to-body ...","1584 x 2160 pixels, 4:3 ratio (~269 ppi density)",Li-Ion 5600 mAh,"200 MP, f/1.7, 24mm (wide), 1/1.3"", 0.6µm, mul...","10 MP, f/2.2, 18mm (ultrawide), 1/3.0"", 1.12µm...","512GB 16GB RAM, 1TB 16GB RAM","$ 2,899.00"
2,Samsung Galaxy M17,"2025, October 10","6.7 inches, 110.2 cm2 (~86.0% screen-to-body r...","1080 x 2340 pixels, 19.5:9 ratio (~385 ppi den...",5000 mAh,"50 MP, f/1.8, (wide), 1/2.76"", 0.64µm, AF, OIS...","13 MP, f/2.0, (wide), 1/3.1"", 1.12µm","128GB 4GB RAM, 128GB 6GB RAM, 128GB 8GB RAM",About 120 EUR
3,Samsung Galaxy F07,"2025, October 04","6.7 inches, 108.4 cm2 (~83.6% screen-to-body r...","720 x 1600 pixels, 20:9 ratio (~262 ppi density)",5000 mAh,"50 MP, f/1.8, (wide), 1/2.76"", 0.64µm, PDAF\nA...","8 MP, f/2.0, (wide), 1/4.0"", 1.12µm",64GB 4GB RAM,£ 189.99
4,Samsung Galaxy M07,"2025, October 01","6.7 inches, 108.4 cm2 (~83.6% screen-to-body r...","720 x 1600 pixels, 20:9 ratio (~262 ppi density)",5000 mAh,"50 MP, f/1.8, (wide), 1/2.76"", 0.64µm, PDAF\nA...","8 MP, f/2.0, (wide), 1/4.0"", 1.12µm",64GB 4GB RAM,"₹ 7,849"


In [52]:
def extract_and_convert_price_to_usd(price_str):

    if pd.isna(price_str):
        return None

    # conversion rates
    conversion_rates = {
        '€': 1.18, 'EUR': 1.18,   # Euro to USD
        '£': 1.36, 'GBP': 1.36,   # British Pound to USD
        '₹': 0.011, 'INR': 0.011  # Indian Rupee to USD
    }

    price_matches = re.findall(
        r'(\$|€|£|₹|EUR|GBP|INR)\s*([\d,\.]+)',
        str(price_str),
        re.IGNORECASE
    )

    converted_prices = []

    for currency_symbol, price_value in price_matches:
        cleaned_price = price_value.replace(',', '')

        try:
            price_float = float(cleaned_price)
            currency_symbol_upper = currency_symbol.upper()

            # if already in USD
            if currency_symbol == '$':
                converted_prices.append(f"{price_float:.2f}")

            # converting other currencies to USD
            elif currency_symbol_upper in conversion_rates:
                usd_value = price_float * conversion_rates[currency_symbol_upper]
                converted_prices.append(f"{usd_value:.2f}")

        except ValueError:
            continue

    # special handling for patterns like "About 500 EUR"
    if not converted_prices:
        about_match = re.search(
            r'About\s+([\d,\.]+)\s+(EUR|GBP|INR)',
            str(price_str),
            re.IGNORECASE
        )

        if about_match:
            price_float = float(about_match.group(1).replace(',', ''))
            currency = about_match.group(2).upper()
            usd_value = price_float * conversion_rates[currency]
            converted_prices.append(f"{usd_value:.2f}")

    if not converted_prices:
        return None
    elif len(converted_prices) == 1:
        return converted_prices[0]
    else:
        # multiple prices joined by "/"
        return '/'.join(converted_prices)


In [53]:
df['Price($)'] = df['Price'].apply(extract_and_convert_price_to_usd)
df

,Model,Release Date,Display Size,Display Res,Battery,Main Camera,Selfie Camera,Memory,Price,Price($)
0,Samsung Galaxy A07,"2026, January 13","6.7 inches, 108.4 cm2 (~83.6% screen-to-body r...","720 x 1600 pixels, 20:9 ratio (~262 ppi density)",Li-Ion 6000 mAh,"50 MP, f/1.8, (wide), PDAF\nAuxiliary lens","8 MP, f/2.0, 26mm (wide), 1/4.0"", 1.12µm","128GB 4GB RAM, 128GB 6GB RAM",About 150 EUR,177.00
1,Samsung Galaxy Z TriFold,"2025, December 02","10.0 inches, 305.4 cm2 (~89.6% screen-to-body ...","1584 x 2160 pixels, 4:3 ratio (~269 ppi density)",Li-Ion 5600 mAh,"200 MP, f/1.7, 24mm (wide), 1/1.3"", 0.6µm, mul...","10 MP, f/2.2, 18mm (ultrawide), 1/3.0"", 1.12µm...","512GB 16GB RAM, 1TB 16GB RAM","$ 2,899.00",2899.00
2,Samsung Galaxy M17,"2025, October 10","6.7 inches, 110.2 cm2 (~86.0% screen-to-body r...","1080 x 2340 pixels, 19.5:9 ratio (~385 ppi den...",5000 mAh,"50 MP, f/1.8, (wide), 1/2.76"", 0.64µm, AF, OIS...","13 MP, f/2.0, (wide), 1/3.1"", 1.12µm","128GB 4GB RAM, 128GB 6GB RAM, 128GB 8GB RAM",About 120 EUR,141.60
3,Samsung Galaxy F07,"2025, October 04","6.7 inches, 108.4 cm2 (~83.6% screen-to-body r...","720 x 1600 pixels, 20:9 ratio (~262 ppi density)",5000 mAh,"50 MP, f/1.8, (wide), 1/2.76"", 0.64µm, PDAF\nA...","8 MP, f/2.0, (wide), 1/4.0"", 1.12µm",64GB 4GB RAM,£ 189.99,258.39
4,Samsung Galaxy M07,"2025, October 01","6.7 inches, 108.4 cm2 (~83.6% screen-to-body r...","720 x 1600 pixels, 20:9 ratio (~262 ppi density)",5000 mAh,"50 MP, f/1.8, (wide), 1/2.76"", 0.64µm, PDAF\nA...","8 MP, f/2.0, (wide), 1/4.0"", 1.12µm",64GB 4GB RAM,"₹ 7,849",86.34
5,Samsung Galaxy A17 4G,"2025, September 18","6.7 inches, 110.2 cm2 (~86.0% screen-to-body r...","1080 x 2340 pixels, 19.5:9 ratio (~385 ppi den...",5000 mAh,"50 MP, f/1.8, (wide), 1/2.76"", 0.64µm, AF, OIS...","13 MP, f/2.0, (wide), 1/3.1"", 1.12µm","128GB 4GB RAM, 128GB 6GB RAM, 128GB 8GB RAM, 2...",$ 157.50 / € 142.89 / £ 108.29,157.50/168.61/147.27
6,Samsung Galaxy Tab A11+,"2025, September 29","11.0 inches, 350.9 cm2 (~80.9% screen-to-body ...","1200 x 1920 pixels, 16:10 ratio (~206 ppi dens...",Li-Po 7040 mAh,"8 MP, AF",5 MP,"128GB 6GB RAM, 256GB 6GB RAM, 256GB 8GB RAM",$ 249.99,249.99
7,Samsung Galaxy Tab A11,"2025, September 12","8.7 inches, 214.9 cm2 (~81.7% screen-to-body r...","800 x 1340 pixels, 5:3 ratio (~179 ppi density)",5100 mAh,"8 MP, AF",5 MP,"64GB 4GB RAM, 128GB 6GB RAM, 128GB 8GB RAM",About 180 EUR,212.40
8,Samsung Galaxy F17,"2025, September 11","6.7 inches, 110.2 cm2 (~86.0% screen-to-body r...","1080 x 2340 pixels, 19.5:9 ratio (~385 ppi den...",Li-Ion 5000 mAh,"50 MP, f/1.8, (wide), 1/2.76"", 0.64µm, AF, OIS...","13 MP, f/2.0, (wide), 1/3.1"", 1.12µm","128GB 4GB RAM, 128GB 6GB RAM, 128GB 8GB RAM","₹ 13,375",147.12
9,Samsung Galaxy S25 FE,"2025, September 04","6.7 inches, 110.2 cm2 (~89.2% screen-to-body r...","1080 x 2340 pixels, 19.5:9 ratio (~385 ppi den...",4900 mAh,"50 MP, f/1.8, 24mm (wide), 1/1.57"", 1.0µm, dua...","12 MP, f/2.2, 26mm (wide), 1/3.2"", 1.12µm","128GB 8GB RAM, 256GB 8GB RAM, 512GB 8GB RAM","$ 449.99 / € 495.00 / £ 460.00 / ₹ 59,999",449.99/584.10/625.60/659.99


In [54]:
df = df.drop(columns=['Price'])

df.columns

Index(['Model', 'Release Date', 'Display Size', 'Display Res', 'Battery',
       'Main Camera', 'Selfie Camera', 'Memory', 'Price($)'],
      dtype='object')

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Model          27 non-null     object
 1   Release Date   27 non-null     object
 2   Display Size   27 non-null     object
 3   Display Res    27 non-null     object
 4   Battery        27 non-null     object
 5   Main Camera    27 non-null     object
 6   Selfie Camera  25 non-null     object
 7   Memory         27 non-null     object
 8   Price($)       27 non-null     object
dtypes: object(9)
memory usage: 2.0+ KB


In [56]:
output_path = '/content/samsung_specs_processed.csv'
df.to_csv(output_path, index=False)

print(f"CSV file successfully saved at: {output_path}")

CSV file successfully saved at: /content/samsung_specs_processed.csv
